In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import date, timedelta

from core.portfolio.risk_manager import RiskManager, RiskConfig

# Refactored utilities (for future use)
from utils.plotter import PortfolioPlotter
from utils.formatter import PerformanceSummary

## 1. Basic Setup

Create RiskManager with default kill switch settings.

In [ ]:
# Create config with kill switches
config = RiskConfig(
    max_drawdown_pct=0.15,      # 15% max drawdown → KILL
    max_daily_loss_pct=0.03,    # 3% daily loss → KILL
    max_portfolio_heat_pct=0.10, # 10% max at risk
    min_capital_pct=0.50        # 50% capital floor → KILL
)

risk_mgr = RiskManager(config)

# Initialize capital tracking
risk_mgr.initialize_capital_tracking(initial_capital=1_000_000)

print(f"Starting capital: ${risk_mgr.initial_capital:,.0f}")
print(f"Peak capital: ${risk_mgr.peak_capital:,.0f}")
print(f"Kill switches armed ✅")

Starting capital: $1,000,000
Peak capital: $1,000,000
Kill switches armed ✅


## 2. Trade Pre-Approval

Before each trade, check if risk limits allow it.

In [ ]:
# Example trade request
trade = {
    'asset': 'ES',
    'size': 5.0,           # 5 contracts
    'price': 4500.0,       # $4500 per contract
    'notional': 22500.0    # $22.5K position value
}

# Check if trade is approved
approved, reasons = risk_mgr.check_trade_approval(
    asset=trade['asset'],
    size=trade['size'],
    price=trade['price'],
    portfolio_value=risk_mgr.initial_capital
)

if approved:
    print(f"✅ Trade APPROVED: {trade['asset']}, {trade['size']} contracts @ ${trade['price']:,.0f}")
    print(f"   Notional value: ${trade['notional']:,.0f}")
else:
    print(f"❌ Trade REJECTED:")
    for reason in reasons:
        print(f"   - {reason}")

✅ Trade APPROVED: ES, 5.0 contracts @ $4,500
   Notional value: $22,500


## 3. Simulate Drawdown Kill Switch

What happens when account drops 15% from peak?

In [ ]:
# Simulate losing day
print("\n📉 Simulating drawdown...\n")

# Day 1: Small loss
risk_mgr.update_capital(current_capital=980_000, current_date=date.today())
print(f"Day 1: Capital = $980K, Drawdown = {risk_mgr.get_current_drawdown(980_000):.2%}")
print(f"Status: {'🔴 KILLED' if risk_mgr.is_killed else '✅ Active'}\n")

# Day 2: Medium loss
risk_mgr.update_capital(current_capital=920_000, current_date=date.today() + timedelta(days=1))
print(f"Day 2: Capital = $920K, Drawdown = {risk_mgr.get_current_drawdown(920_000):.2%}")
print(f"Status: {'🔴 KILLED' if risk_mgr.is_killed else '✅ Active'}\n")

# Day 3: Large loss → KILL SWITCH
risk_mgr.update_capital(current_capital=840_000, current_date=date.today() + timedelta(days=2))
print(f"Day 3: Capital = $840K, Drawdown = {risk_mgr.get_current_drawdown(840_000):.2%}")
print(f"Status: {'🔴 KILLED' if risk_mgr.is_killed else '✅ Active'}")

if risk_mgr.is_killed:
    print(f"\n⚠️ KILL REASON: {risk_mgr.kill_reason}")
    print(f"Breach history: {len(risk_mgr.breach_history)} violations logged")


📉 Simulating drawdown...

Day 1: Capital = $980K, Drawdown = 2.00%
Status: ✅ Active

Day 2: Capital = $920K, Drawdown = 8.00%
Status: ✅ Active


🚨 KILL SWITCH ACTIVATED 🚨
Reason: Max drawdown 16.0% reached (limit: 15.0%)
Time: 2025-11-29 15:52:01.369507
Day 3: Capital = $840K, Drawdown = 16.00%
Status: 🔴 KILLED

⚠️ KILL REASON: Max drawdown 16.0% reached (limit: 15.0%)
Breach history: 1 violations logged


## 4. Daily Loss Kill Switch

What happens when account loses 3% in one day?

In [ ]:
# Reset for new test
risk_mgr = RiskManager(config)
risk_mgr.initialize_capital_tracking(initial_capital=1_000_000)

print("\n📉 Simulating daily loss kill switch...\n")

# Start of day: $1M
print(f"Day start: $1,000,000")

# Mid-day: Down 2.5% (still OK)
risk_mgr.update_capital(current_capital=975_000, current_date=date.today())
print(f"Mid-day: $975K (Daily PnL: {risk_mgr.get_daily_pnl_pct(975_000):.2%}) - Status: {'🔴 KILLED' if risk_mgr.is_killed else '✅ Active'}")

# End of day: Down 3.5% → KILL SWITCH
risk_mgr.update_capital(current_capital=965_000, current_date=date.today())
print(f"End of day: $965K (Daily PnL: {risk_mgr.get_daily_pnl_pct(965_000):.2%}) - Status: {'🔴 KILLED' if risk_mgr.is_killed else '✅ Active'}")

if risk_mgr.is_killed:
    print(f"\n⚠️ KILL REASON: {risk_mgr.kill_reason}")


📉 Simulating daily loss kill switch...

Day start: $1,000,000
Mid-day: $975K (Daily PnL: -2.50%) - Status: ✅ Active

🚨 KILL SWITCH ACTIVATED 🚨
Reason: Daily loss -3.5% reached (limit: -3.0%)
Time: 2025-11-29 15:52:25.008697
End of day: $965K (Daily PnL: -3.50%) - Status: 🔴 KILLED

⚠️ KILL REASON: Daily loss -3.5% reached (limit: -3.0%)


## 5. Risk Status Report

Print comprehensive risk metrics.

In [ ]:
# Create active risk manager
risk_mgr = RiskManager(config)
risk_mgr.initialize_capital_tracking(initial_capital=1_000_000)
risk_mgr.update_capital(current_capital=950_000, current_date=date.today())

# Print status
risk_mgr.print_risk_status()


🚨 KILL SWITCH ACTIVATED 🚨
Reason: Daily loss -5.0% reached (limit: -3.0%)
Time: 2025-11-29 15:53:12.787313

📊 RISK STATUS
🚨 KILL SWITCH: ACTIVE
   Reason: Daily loss -5.0% reached (limit: -3.0%)

⚠️  Risk Limits:
   Max Drawdown:     15.0%
   Max Daily Loss:   3.0%
   Max Position:     20.0%
   Max Heat:         10.0%


## 6. Integration Example

How to use RiskManager in your backtest/live trading loop.

In [ ]:
# Pseudo-code for integration
print("""
# In your trading loop:

# 1. Start of day
if risk_mgr.initial_capital is None:
    risk_mgr.initialize_capital_tracking(initial_capital=portfolio.equity)

# 2. Before each trade
approved, reasons = risk_mgr.check_trade_approval(
    asset=signal.asset,
    size=signal.size,
    price=current_price,
    current_positions=portfolio.positions,
    portfolio_value=portfolio.equity
)

if not approved:
    logger.warning(f"Trade rejected: {reasons}")
    continue  # Skip this trade

# 3. Execute trade
portfolio.execute_trade(signal)

# 4. After each trade / end of day
risk_mgr.update_capital(
    current_capital=portfolio.equity,
    current_date=current_date
)

# 5. Check if killed
if risk_mgr.is_killed:
    logger.critical(f"KILL SWITCH TRIGGERED: {risk_mgr.kill_reason}")
    send_alert_to_trader()
    break  # Stop trading
""")


# In your trading loop:

# 1. Start of day
if risk_mgr.initial_capital is None:
    risk_mgr.initialize_capital_tracking(initial_capital=portfolio.equity)

# 2. Before each trade
approved, reasons = risk_mgr.check_trade_approval(
    asset=signal.asset,
    size=signal.size,
    price=current_price,
    current_positions=portfolio.positions,
    portfolio_value=portfolio.equity
)

if not approved:
    logger.warning(f"Trade rejected: {reasons}")
    continue  # Skip this trade

# 3. Execute trade
portfolio.execute_trade(signal)

# 4. After each trade / end of day
risk_mgr.update_capital(
    current_capital=portfolio.equity,
    current_date=current_date
)

# 5. Check if killed
if risk_mgr.is_killed:
    logger.critical(f"KILL SWITCH TRIGGERED: {risk_mgr.kill_reason}")
    send_alert_to_trader()
    break  # Stop trading



## Summary

The existing **RiskManager** class provides:

✅ **Kill Switches**:
- Max drawdown (default: 15%)
- Daily loss limit (default: 3%)
- Minimum capital floor (default: 50%)

✅ **Monitoring**:
- Capital tracking (initial, peak, daily)
- Breach history logging
- Portfolio heat calculations

✅ **Pre-Trade Validation**:
- `check_trade_approval()` before execution
- Automatic rejection if kill switch active

✅ **Status Reporting**:
- `print_risk_status()` for debugging
- Real-time drawdown/loss metrics

**Next Steps**: Integrate RiskManager into your backtest engine and live trading system.